In [1]:
import pandas as pd
import numpy as np

### 1. Prepare train and test datasets:

In [2]:
spams = pd.read_table("SMSSpamCollection.txt",sep="\t",header=0)

In [3]:
spams

,classe,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
messages = pd.read_table("SMSSpamCollection.txt",sep="\t")
messages

,classe,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
spams.classe.value_counts()

ham     4825
spam     747
Name: classe, dtype: int64

In [6]:
# Split data
from sklearn.model_selection import train_test_split
spamsTrain, spamsTest = train_test_split(spams,train_size=3572,random_state=1,stratify=spams['classe'])

In [7]:
#frequency distribution of the class attribute
#train set
freqTrain = pd.crosstab(index=spamsTrain["classe"],columns="count")
print(freqTrain/freqTrain.sum())
#test set
freqTest = pd.crosstab(index=spamsTest["classe"],columns="count")
print(freqTest/freqTest.sum())

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

col_0      count
classe          
ham     0.865901
spam    0.134099
col_0   count
classe       
ham     0.866
spam    0.134


### 2. Text pre-processing:

In [9]:
import re
import nltk
nltk.download('stopwords')
#used once to download stopwords.
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0,len(messages['message'])):
    smsDocument = re.sub('[^a-zA-Z]',' ',messages['message'][i])
    smsDocument = smsDocument.lower()
    smsDocument = smsDocument.split()
    ps = PorterStemmer()
    smsDocument = [ps.stem(word) for word in smsDocument if not word in set(stopwords.words('english'))]
    smsDocument = ' '.join(smsDocument)
    corpus.append(smsDocument)
smsDocuments = pd.Series(corpus)

[nltk_data] Downloading package stopwords to C:\Users\jannett
[nltk_data]     chabbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

### 3. Feature engineering

bag of words

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
parseur = CountVectorizer()
#create the document term matrix
XTrain = parseur.fit_transform(spamsTrain['message'])
#number of tokens
print(len(parseur.get_feature_names()))
#list of tokens
print(parseur.get_feature_names())

6789
['00', '000', '000pes', '0089', '01223585236', '01223585334', '0125698789', '02', '0207', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07046744435', '07090298926', '07099833605', '07123456789', '0721072', '07732584351', '07734396839', '07753741225', '0776xxxxxxx', '07781482378', '07786200117', '077xxx', '07808247860', '07821230901', '07880867867', '07946746291', '07973788240', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767', '08001950382', '08002888812', '08002986906', '08006344447', '0808', '08081263000', '08081560665', '0825', '083', '08448714184', '0845', '08452810073', '08452810075over18', '0870', '08700435505150p', '08700469649', '08700621170150p', '08701213186', '08701237397', '08701417012', '087016248', '08701752560', '087018728737', '0870241182716', '08702840625', '08704439680', '08704439680ts', '08706091795', '0870737910216yrs', '08707500020', '08707509020', '0870753331018', '08707808226', 

In [25]:
mdtTrain = XTrain.toarray()
#type of the matrix
print(type(mdtTrain))
#size of the matrix
print(mdtTrain.shape)
#frequency of the terms
freq_mots = np.sum(mdtTrain,axis=0)
print(freq_mots)
#arg sort
index = np.argsort(freq_mots)
print(index)

<class 'numpy.ndarray'>
(3572, 6789)
[ 7 17  1 ...  1  1  1]
[3394 3804 3803 ... 5962 6061 6756]


In [24]:
mdtTrain

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
#print the terms and their frequency
imp = {'terme':np.asarray(parseur.get_feature_names())[index],'freq':freq_mots[index]}
print(pd.DataFrame(imp))

           terme  freq
0          kaiez     1
1        mandara     1
2          manda     1
3     manchester     1
4         manage     1
...          ...   ...
6784          in   584
6785         and   645
6786         the   831
6787          to  1439
6788         you  1495

[6789 rows x 2 columns]


TF-IDF Model

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(mdtTrain)
print ("IDF:",tfidf.idf_)
tf_idf_matrix = tfidf.transform(mdtTrain)
print (tf_idf_matrix.todense())
print (tf_idf_matrix.todense().shape)

IDF: [7.10171932 6.34794751 8.48801368 ... 8.48801368 8.48801368 8.48801368]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(3572, 6789)


### 4. Model training and prediction (Classification):

In [20]:
#*** train the classifier
#import the class Logistic Regression
from sklearn.linear_model import LogisticRegression
#instatiate the object
modelFirst = LogisticRegression()
#perform the training process
modelFirst.fit(mdtTrain,spamsTrain['classe'])
#size of coefficients matrix
print(modelFirst.coef_.shape)
#intercept
print(modelFirst.intercept_)
#generate the document term matrix for the test set
#using the object learned from the train set
#create the document term matrix
mdtTest = parseur.transform(spamsTest['message'])
#size of the matrix
print(mdtTest.shape)
#prediction for the test set
predTest = modelFirst.predict(mdtTest)

c:\users\jannett chabbeh\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(1, 6789)
[-4.41615423]
(2000, 6789)


### Model evaluation

In [21]:
#import the metrics class for the performance measurement
from sklearn import metrics

#confusion matrix
mcTest = metrics.confusion_matrix(spamsTest['classe'],predTest)
print(mcTest)
#recall
print(metrics.recall_score(spamsTest['classe'],predTest,pos_label='spam'))
#precision
print(metrics.precision_score(spamsTest['classe'],predTest,pos_label='spam'))
#accuracy rate
print(metrics.accuracy_score(spamsTest['classe'],predTest))
#F1-Score
print(metrics.f1_score(spamsTest['classe'],predTest,pos_label='spam'))

[[1729    3]
 [  39  229]]
0.8544776119402985
0.9870689655172413
0.979
0.9159999999999999
